In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler#para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test
import pandas as pd
from sklearn.utils import shuffle
from ast import literal_eval
import time
import sys
import os

from functions.auxiliares import PaddingSameSize, MatrixFormat_To_Vector, RemoveOversizedMatrix, StandardSize_Padding, CNN_Features_Format, batch_format


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Esto garantiza que se ejecutara en GPU si esta disponible
print(device)

cpu


In [3]:
max_r = 40 ## Maximo numero de subconsultas
max_c = 50 ## Maximo numero de subcarateristicas

In [4]:
df_raw = pd.read_csv('D:/.Memoria/Test/Jupyter/test_example.csv')
columns = list(df_raw.columns)
print(columns)
#  Eliminar los con tiempos de ejecución 0 o muy altos
df_clean = shuffle(df_raw[(df_raw['ql_rt_msec'] > 0) & (df_raw['ql_rt_msec'] < 1e3)])
df_clean.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean.csv',index=False)

### HAY QUE CONVERTIR ALGUNAS COLUMNAS Q ESTAN EN PORCENTAJE A FLOAT
df_clean['ql_rt_clocks'] = df_clean['ql_rt_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_seg'] = df_clean['ql_same_seg'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_page'] = df_clean['ql_same_page'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_cl_wait_clocks'] = df_clean['ql_cl_wait_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_clocks'] = df_clean['ql_c_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_cl_wait'] = df_clean['ql_c_cl_wait'].apply(lambda x: float(x.strip('%'))/100)

# Convertir MatrixFormat de STR a np.array
df_clean['matrix_format'] = df_clean['matrix_format'].apply(lambda x: np.asarray(literal_eval(x)).astype(np.float32)) 

## shape de raw y clean
print(f'shape df_raw: {df_raw.shape}')
print(f'shape df_clean: {df_clean.shape}')


new_df_clean,max_new_r,max_new_c,min_new_r,min_new_c,mean_new_r,mean_new_c = RemoveOversizedMatrix(df_clean,max_r,max_c)

#num_standard_rows = max_new_r
#num_standard_columns = max_new_c
num_standard_rows = 20
num_standard_columns = 36

new_df_clean['matrix_format'] = df_clean['matrix_format'].apply(lambda x: StandardSize_Padding(x,num_standard_rows,num_standard_columns)) 

print(f'shape new_df_clean: {new_df_clean.shape}')

msk = np.random.rand(len(new_df_clean)) <= 0.8
df_train = new_df_clean[msk]
df_test = new_df_clean[~msk]

df_train.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_train.csv',index=False)
df_test.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_test.csv',index=False)



['unique_id', 'filename', 'sparql_file', 'profile', 'limit', 'ql_rt_msec', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait', 'matrix_format', 'binary_tree']
shape df_raw: (3711, 21)
shape df_clean: (2156, 21)
Total df_clean:  2156
Eliminados por Oversized: 31
shape new_df_clean: (2125, 21)


In [5]:
features = [
    'matrix_format'
] 

## FEATURES - TRAIN Y TEST
X_df_train = df_train[features]
X_df_test = df_test[features]
X_numpy_train = X_df_train.to_numpy()
X_numpy_test = X_df_test.to_numpy()



## TARGETS - TRAIN Y TEST
y_df_train = df_train['ql_rt_msec']
y_df_test = df_test['ql_rt_msec']

y_numpy_train = y_df_train.to_numpy().astype(np.float32)
y_numpy_test = y_df_test.to_numpy().astype(np.float32)


print("-----------------------")
print(f'shape X_numpy_train: {X_numpy_train.shape}')
print(f'shape X_numpy_test: {X_numpy_test.shape}')
print(f'shape y_numpy_train: {y_numpy_train.shape}')
print(f'shape y_numpy_test: {y_numpy_test.shape}')
print("-----------------------")


# Pasarlos a Torch. 
#X_train = torch.from_numpy(X_numpy_train)
#X_test = torch.from_numpy(X_numpy_test)

#X_train = torch.tensor(df_train['matrix_format'].values)
#X_test = torch.tensor(df_test['matrix_format'].values)


X_train = CNN_Features_Format(X_numpy_train)
X_test = CNN_Features_Format(X_numpy_test)

y_train = torch.from_numpy(y_numpy_train)
y_test = torch.from_numpy(y_numpy_test)


# También pasar los targets de vector fila a vector columna
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


print(f'shape X_train: {X_train.shape}')
print(f'shape X_test: {X_test.shape}')
print(f'shape y_train: {y_train.shape}')
print(f'shape y_test: {y_test.shape}')
print("-----------------------")

-----------------------
shape X_numpy_train: (1699, 1)
shape X_numpy_test: (426, 1)
shape y_numpy_train: (1699,)
shape y_numpy_test: (426,)
-----------------------
shape X_train: torch.Size([1699, 1, 20, 36])
shape X_test: torch.Size([426, 1, 20, 36])
shape y_train: torch.Size([1699, 1])
shape y_test: torch.Size([426, 1])
-----------------------


## Parametros

In [6]:
#n_samples_train, n_features_train = X_train.shape
#n_samples_test, n_features_test = X_test.shape
#
#hidden_size = 100000
#input_size = n_features_train 
#print(f'n_samples_train: {n_samples_train}')
#print(f'n_features_train: {n_features_train}')
#print(f'n_samples_test: {n_samples_test}')
#print(f'n_features_test: {n_features_test}')
#print(f'learning_rate: {learning_rate}')
#print(f'hidden_size: {hidden_size}')
#print(f'input_size: {input_size}')
learning_rate = 0.001
batches_size = 100
num_epochs = 10000
num_batches = math.ceil(X_train.shape[0]/batches_size)
batches = batch_format(X_train,y_train,batches_size)
for i in range(num_batches):
    print(batches["features"][i].shape)
    print(batches["labels"][i].shape)
print(X_train.shape)
print(y_train.shape)

torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([100, 1, 20, 36])
torch.Size([100, 1])
torch.Size([99, 1, 20, 36])
torch.Size([99, 1])
torch.Size([1699, 1, 20, 36])
torch.Size([1699, 1])


## Modelo

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # SE DEFINEN LAS CAPAS A UTILIZAR, TANTO LAS CNN, FCL-ANN CY LAS DE POOLING
        self.conv1 = nn.Conv2d(1, 6, (3,5),stride=(1,1)) 
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(6, 12, (3,5),stride=(1,1)) 
        self.fc1 = nn.Linear(12*3*6, 162)
        self.fc2 = nn.Linear(162, 84)
        self.fc3 = nn.Linear(84, 42)
        self.fc4 = nn.Linear(42, 20)
        self.fc5 = nn.Linear(20, 1)
        
        #Se pueden agregar mas capas o mas neurones, cambiar tamaños etc.. pero siempre debo terminar con
        # una salida del tamaño que busco

    def forward(self, x):
        #print("input ",x.shape)
        x = F.leaky_relu(self.conv1(x), negative_slope=0.01, inplace=False)
        #print("conv1 ",x.shape)
        x = self.pool(x)
        #print("pooling ",x.shape)
        
        
        #print("input ",x.shape)
        x = F.leaky_relu(self.conv2(x), negative_slope=0.01, inplace=False)
        #print("conv2 ",x.shape)
        x = self.pool(x)
        #print("pooling2 ",x.shape)
        
        
        x = x.view(-1,12*3*6)           
        #print("view: ",x.shape)
        x = F.leaky_relu(self.fc1(x), negative_slope=0.01, inplace=False)
        #print("fc1: ",x.shape)
        x = F.leaky_relu(self.fc2(x), negative_slope=0.01, inplace=False)
        #print("fc2: ",x.shape)
        x = F.leaky_relu(self.fc3(x), negative_slope=0.01, inplace=False)
        #print("fc3: ",x.shape)
        x = F.leaky_relu(self.fc4(x), negative_slope=0.01, inplace=False)
        #print("fc4: ",x.shape)
        x =self.fc5(x)
        #print("fc5: ",x.shape)
        #print("---------------------------------------------------------------")
        return x


model = ConvNet().to(device)

## Loss y Optimizer

In [8]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

## Training Loop

In [9]:
#print(num_batches)
for epoch in range(num_epochs):
    for bt in range(num_batches):
        #print(batches["features"][bt].shape)
        #print(batches["labels"][bt].shape)
        y_hat = model(batches["features"][i])
        loss = criterion(y_hat,batches["labels"][i])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if bt+1 == num_batches and (epoch+1) % 100 == 0:
            print(f'epoch: [{epoch+1}/{num_epochs}], loss = {loss.item()}')
            
        

epoch: [100/10000], loss = 1636.033203125
epoch: [200/10000], loss = 815.4194946289062
epoch: [300/10000], loss = 25.884057998657227
epoch: [400/10000], loss = 16.920761108398438
epoch: [500/10000], loss = 25928.912109375
epoch: [600/10000], loss = 22183.037109375
epoch: [700/10000], loss = 9931.1708984375
epoch: [800/10000], loss = 3121.679931640625
epoch: [900/10000], loss = 340.4971618652344
epoch: [1000/10000], loss = 63.6439208984375
epoch: [1100/10000], loss = 950.1453857421875
epoch: [1200/10000], loss = 832.1869506835938
epoch: [1300/10000], loss = 804.499267578125
epoch: [1400/10000], loss = 779.28955078125
epoch: [1500/10000], loss = 712.470458984375
epoch: [1600/10000], loss = 16605.091796875
epoch: [1700/10000], loss = 12500.5859375
epoch: [1800/10000], loss = 1037.5615234375
epoch: [1900/10000], loss = 651.7359008789062
epoch: [2000/10000], loss = 319.89947509765625
epoch: [2100/10000], loss = 288.06719970703125
epoch: [2200/10000], loss = 282.5764465332031
epoch: [2300/10

### Testing

In [10]:
# BASICO
print(X_test.shape)
print(y_test.shape)
print(batches_size)



torch.Size([426, 1, 20, 36])
torch.Size([426, 1])
100


In [11]:

batches_size_test = 5
num_batches_test = math.ceil(X_test.shape[0]/batches_size_test)
batches_test = batch_format(X_test,y_test,batches_size_test)
#for i in range(num_batches_test):
#    print(batches_test["features"][i].shape)
#    print(batches_test["labels"][i].shape)
#print(X_test.shape)
#print(y_test.shape)

y_pred = model(batches_test["features"][0])
y = batches_test["labels"][0]
print(y_pred)

print(y)

print()



tensor([[57.4295],
        [31.2158],
        [22.5894],
        [20.7062],
        [ 8.3708]], grad_fn=<AddmmBackward>)
tensor([[1.],
        [4.],
        [1.],
        [3.],
        [4.]])



In [12]:
tol = 100 # milisegundos de tolerancia
num_aciertos = 0
num_test = float(y_test.shape[0])
with torch.no_grad():
    for bt in range(num_batches_test):
        #print(batches["features"][bt].shape)
        #print(batches["labels"][bt].shape)
        y_pred_tensor = model(batches_test["features"][bt])
        y_tensor = batches_test["labels"][bt]
        for i in range(y_tensor.shape[0]):
            y_pred = y_pred_tensor[i].item()
            y = y_tensor[i].item()
            print(y, y_pred)
            if abs(y_pred-y) < tol:
                num_aciertos += 1 
            
    accuracy = num_aciertos/num_test

print(f'achuntes: [{num_aciertos}/{num_test}]')
print(f'accuracy: {accuracy*100}')
      
      

            

1.0 57.42951202392578
4.0 31.215810775756836
1.0 22.589359283447266
3.0 20.706201553344727
4.0 8.370789527893066
2.0 320.5621643066406
44.0 92935.34375
10.0 22.023801803588867
2.0 1.0915131568908691
1.0 23.834793090820312
33.0 -10.315242767333984
2.0 8.28476619720459
1.0 1.2791361808776855
11.0 82.8336181640625
5.0 2.7809224128723145
3.0 3.4349470138549805
15.0 24.00014305114746
1.0 9.619932174682617
59.0 38.67776107788086
3.0 2.755800247192383
1.0 25.013633728027344
52.0 53.96715545654297
386.0 8.664472579956055
263.0 -3.5603909492492676
2.0 1.1949982643127441
22.0 0.4723701477050781
2.0 0.7541012763977051
1.0 -12.968559265136719
4.0 7.981889247894287
6.0 8.063091278076172
1.0 456.29461669921875
3.0 5.5072503089904785
1.0 0.974769115447998
1.0 1.0754036903381348
1.0 9.772748947143555
83.0 31.99833106994629
2.0 8.445605278015137
2.0 8.399554252624512
1.0 0.6466431617736816
448.0 -3.7841315269470215
3.0 0.9272513389587402
3.0 1.0908203125
2.0 3.703005313873291
22.0 7.273335933685303
4.0

5.0 3.5894341468811035
141.0 47.18703079223633
2.0 1.0715570449829102
7.0 19.195083618164062
3.0 8.54167366027832
5.0 6.022917747497559
4.0 4.864897727966309
2.0 0.44577550888061523
947.0 210.99253845214844
174.0 308.4110107421875
1.0 0.4557523727416992
2.0 7.735346794128418
213.0 8.386078834533691
5.0 -0.14954805374145508
2.0 4.242612838745117
1.0 -7.475252628326416
1.0 8.396169662475586
277.0 6.808466911315918
11.0 7.574275016784668
3.0 4.890743732452393
1.0 29.133302688598633
808.0 -3.696981906890869
2.0 7.6142401695251465
25.0 16.371665954589844
6.0 -5.740767955780029
2.0 -20.60934829711914
18.0 5.9705810546875
3.0 7.9905195236206055
4.0 0.7893390655517578
457.0 4.083683013916016
4.0 8.09863567352295
219.0 3.429037570953369
2.0 2.6473398208618164
16.0 7.60629415512085
4.0 8.255711555480957
20.0 353.0633239746094
16.0 7.057888031005859
70.0 0.6502056121826172
25.0 6.904642581939697
6.0 6.453691005706787
47.0 32.26020050048828
5.0 -2.04264497756958
3.0 1.0618553161621094
5.0 0.790521